In [ ]:
#Read the .txt file, check the glomeruli centroid coordinates and store 1 if it is in the medulla and 
#0 otherwise.

#Path to the glomeruli .txt file
txtFilePath = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on small image/Kidney3_derecho/glomeruliData-afterZfilterAndConComps.txt'

outputTxtFilePath = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on small image/Kidney3_derecho/glomeruliData-medullaInOut.txt'

medullaSegPath = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole' + \
' image/Result on small image/MedullaSegmentation_Kidney3_derecho.tif'

sliceToShow=35


#--------------------------------------------------------------------------
import numpy as np
import tifffile as tiff

medImage = tiff.memmap(medullaSegPath)

if np.max(medImage)!=1:
    medImage = medImage/np.max(medImage)

medImage = medImage.astype(int)
    
dataGloms = open(txtFilePath,"r+")

headers = dataGloms.readline().split('\t')
headers[-1]=headers[-1].split('\n')[0]
headers.append('medulla')

means = dataGloms.readline()
stdevs = dataGloms.readline()


dataStruct = dataGloms.readlines()

dataGloms.close()

print(headers)

outputTxt = open(outputTxtFilePath,'w')
outputTxt.write('\t'.join(headers)+'\n')

numMedulla=0
for line in dataStruct[:-1]:
    lineComps = line.split('\t')
    lineComps[-1]=lineComps[-1].split('\n')[0]
    #print(len(lineComps))
    #print(len(headers))
    #Obtain the centroid coordinates in the same column as the position of "centroid" in the header.
    centroidCoords = np.asarray(lineComps[headers.index('centroid')][1:-1].split(','),dtype=float)
    #print(centroidCoords)
    #Round the coordinates to integer values
    centroidCoords = np.around(centroidCoords,decimals=0).astype(int)
    #print(centroidCoords)
    
    medullaValue = medImage[centroidCoords[0],centroidCoords[1],centroidCoords[2]]
    #print('MedullaValue: ',medullaValue)

    numMedulla+=medullaValue
    
    #Add the medulla value to line    
    line = '\t'.join(lineComps) + '\t'  + str(medullaValue)
    print(line)
    print('\n')
    outputTxt.write(line+'\n')
outputTxt.close()
print('numMedulla: ',numMedulla)
print('Total number of glomeruli: ',len(dataStruct))